In [1]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
def generate_dataset(n):
    x = np.arange(n)
    res_x = []
    res_y = []
    for i in range(n):
        r = 0.15 if i < n/2 else 0.75
        y_i = 1 if np.random.rand() < r else 0
        res_x.append([1, x[i]])
        res_y.append(y_i)
    return np.array(res_x), np.array(res_y)

In [3]:
def proba(coef, x):
    return 1 / (1+np.exp(-coef.dot(x.T)))

In [4]:
def cross_entropy(coef, x, y):
    p = proba(coef, x)
    return -np.mean(y*np.log(p) + (1-y)*np.log(1-p))

In [5]:
def gradient(coef, x, y):
    e = np.exp(coef.dot(x.T))
    return -np.mean(x.T * (y - 1 + 1/(1+e)), axis=1)

In [6]:
def logistic_regression(coef, x, y, lr, b1=0.9, b2=0.999, epsilon=1e-8):
    prev_error = 0
    m_coef = np.zeros(coef.shape)
    v_coef = np.zeros(coef.shape)
    moment_m_coef = np.zeros(coef.shape)
    moment_v_coef = np.zeros(coef.shape)
    t = 0
    while True:
        error = cross_entropy(coef, x, y)
        if abs(error - prev_error) <= epsilon:
            break
        prev_error = error
        grad = gradient(coef, x, y)
        t += 1
        m_coef = b1*m_coef + (1-b1)*grad
        v_coef = b2*v_coef + (1-b2)*grad**2
        moment_m_coef = m_coef / (1-b1**t)
        moment_v_coef = v_coef / (1-b2**t)
        delta = (lr / moment_v_coef**0.5 + 1e-8) * (b1*moment_m_coef + (1-b1)*grad/(1-b1**t))
        coef = np.subtract(coef, delta)
    return coef

In [7]:
coef = np.array([0, 0])
x, y = generate_dataset(200)
p = figure()
p.scatter(x[:,1], y)
show(p)

In [8]:
coef_estimation = logistic_regression(coef, x, y, 1e-3, epsilon=1e-10)

In [9]:
p = figure()
p.scatter(x[:,1], y)
p.line(x[:,1], proba(coef_estimation, x), color="orange")
show(p)
coef_estimation

array([-2.5265281 ,  0.02155576])